In [ ]:
!pip install spacy
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 18.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Actividad 1: Análisis humano


| Caso | ¿Quién depende de quién? | Significado Lógico |
| --- | --- | --- |
| **Caso A (Instrumento)** | "con el telescopio" depende de **"Vi"** | El telescopio es la herramienta del observador. |
| **Caso B (Posesión)** | "con el telescopio" depende de **"hombre"** | El hombre es quien sostiene o tiene el telescopio. |


```
Caso A:
         [Vi]
        /    \
[al hombre] [con el telescopio]
```

```
Caso B:
           [Vi]
            |
        [al hombre]
            |
      [con el telescopio]
```

In [ ]:
import spacy
from spacy import displacy

# Cargamos el modelo
nlp = spacy.load("es_core_news_md")

# Frase original
texto = "Vi al hombre con el telescopio"
doc = nlp(texto)

# Visualizar dependencias
print(f"Interpretacion por defecto para: '{texto}'")
displacy.render(doc, style="dep", options={'distance': 100})

# Iterar tokens para ver el 'head'
for token in doc:
    if token.text == "telescopio":
      print(f"--> El 'telescopio' modifica a: {token.head.text} ({token.dep_})")

Interpretacion por defecto para: 'Vi al hombre con el telescopio'


--> El 'telescopio' modifica a: Vi (obl)


# Preguntas:

 * El verbo 'ver' segun nuestro modelo es el que resulta modificado por poarte de 'telescopio'

 * Utilizaria una frase tal como : 'Vi al hombre que tenía el telescopio'.
  De esta forma 'telescopio'  modifica a 'tenía'

# Actividad 2: Analogias Vectoriales

In [ ]:
import numpy as np

def calcular_analogia(palabra_a, palabra_b, palabra_c, palabra_d):
    token_a = nlp(palabra_a)
    token_b = nlp(palabra_b)
    token_c = nlp(palabra_c)
    token_d = nlp(palabra_d)

    # Similitud directa entre las parejas
    similitud_1 = token_a.similarity(token_b)
    similitud_2 = token_c.similarity(token_d)

    print(f"Pareja 1: {palabra_a} -> {palabra_b}  (Similitud: {similitud_1})")
    print(f"Pareja 2: {palabra_c} -> {palabra_d}  (Similitud: {similitud_2})")

    # Aritmetrica vectorial: A - B + D ~ C
    # (Ej: Paris - Francia + España ~ Madrid)
    vector_resultado = token_a.vector - token_b.vector + token_d.vector

    # Comparamos el vector resultante con la palabra objetivo (C)
    similitud_aritmetica = np.dot(vector_resultado, token_c.vector) / (
        np.linalg.norm(vector_resultado) * np.linalg.norm(token_c.vector)
    )

    print(f"Consistencia analogica (A-B+D ~ C): {similitud_aritmetica:.4f}\n")

# --- ANALISIS CAPITALES ---
print("--- ANALISIS CAPITALES ---")
calcular_analogia("París", "Francia", "Madrid", "España")

# --- ANALISIS OPUESTOS ---
print("--- ANALISIS OPUESTOS ---")
calcular_analogia("Grande", "Pequeño", "Alto", "Bajo")

--- ANALISIS CAPITALES ---
Pareja 1: París -> Francia  (Similitud: 0.6184197664260864)
Pareja 2: Madrid -> España  (Similitud: 0.6086763143539429)
Consistencia analogica (A-B+D ~ C): 0.6936

--- ANALISIS OPUESTOS ---
Pareja 1: Grande -> Pequeño  (Similitud: 0.4286979138851166)
Pareja 2: Alto -> Bajo  (Similitud: 0.43721362948417664)
Consistencia analogica (A-B+D ~ C): 0.5001



# REFLEXIÓN TEÓRICA

* Que el resultado no sea exactamente 1 se debe a que por ejemplo en el caso de Paris - Francia, Paris no esta ligado a Francia como capital, sino tambien como moda, distintos eventos historicos, deporte, todo esto hace que al compararlo con Madrid - España no sea 100% preciso ya que la moda, la historia o los deportes en Madrid son totalemnte distintos a los de Paris.

* El sesgo del corpus se da cuando el corpus (que es la parte con la que hemos entrenado el modelo) tiene mucho mas contexto dobre Paris que sobre Madrid, entonces ahi lo que estaría sucediendo es que los vectores tendran distos pesos y relaciones
